In [3]:
import numpy as np
import pandas as pd
from scipy.linalg import cholesky
from scipy.interpolate import CubicSpline
from scipy.stats import norm
from scipy.optimize import curve_fit
pd.options.mode.chained_assignment = None

Monte Carlo Simulation

In [2]:
def Monte_Carlo(vol_surf, rf_rate, div_rate):
    # Initialise parameters - as of 1-Sep-23
    NKY = 32710.62
    SPX = 4515.77
    HSI = 18382.06
    T = 2

    # Correlation matrix - as of 1-Sep-22
    rho = np.array([[1, 0.187, 0.421],
                    [0.187, 1, 0.11],
                    [0.421, 0.11, 1]])

    # Autocallable parameters
    note_D = 10000  # note denomination
    max_CR = 0.1  # minimum coupon rate
    min_CR = 0.05  # maximum coupon rate
    KO = 1.1  # knock-out price
    KI = 0.5  # knock-in price
    obs_days = 126  # observation every X days
    target = 0.98  # target note price (%)

    # Monte Carlo specific parameters
    N = T * 252  # discrete time steps
    M = 1000  # number of simulations
    dt = T / N

    # Perform (lower) cholesterol decomposition
    lower_chol = cholesky(rho, lower=True)

    # Generate correlated Wiener variables
    W = np.random.normal(0, 1, size=(N, M, 3))
    Z = W @ lower_chol.T

    # Monte Carlo Simulation
    St1 = np.full(shape=(N + 1, M), fill_value=NKY)
    St2 = np.full(shape=(N + 1, M), fill_value=SPX)
    St3 = np.full(shape=(N + 1, M), fill_value=HSI)

    for j in range(1, N + 1):
        r_NKY = rf_rate('NKY', j)  # risk-free rate NKY
        r_SPX = rf_rate('SPX', j)  # risk-free rate SPX
        r_HSI = rf_rate('HSI', j)  # risk-free rate HSI
        q_NKY = div_rate('NKY', j)  # dividend rate NKY
        q_SPX = div_rate('SPX', j)  # dividend rate SPX
        q_HSI = div_rate('HSI', j)  # dividend rate HSI
        vol_NKY = vol_surf('NKY', np.log(NKY/St1[j-1]), j)  # volatility rate NKY
        vol_SPX = vol_surf('SPX', np.log(SPX/St2[j-1]), j)  # volatility rate SPX
        vol_HSI = vol_surf('HSI', np.log(HSI/St3[j-1]), j)  # volatility rate HSI

        # Precompute constants
        nu1dt = ((r_NKY - q_NKY) - 0.5 * vol_NKY ** 2) * dt
        nu2dt = ((r_SPX - q_SPX) - 0.5 * vol_SPX ** 2) * dt
        nu3dt = ((r_HSI - q_HSI) - 0.5 * vol_HSI ** 2) * dt
        vol1sdt = vol_NKY * np.sqrt(dt)
        vol2sdt = vol_SPX * np.sqrt(dt)
        vol3sdt = vol_HSI * np.sqrt(dt)

        St1[j] = St1[j - 1] * np.exp(nu1dt + vol1sdt * Z[j - 1, :, 0])
        St2[j] = St2[j - 1] * np.exp(nu2dt + vol2sdt * Z[j - 1, :, 1])
        St3[j] = St3[j - 1] * np.exp(nu3dt + vol3sdt * Z[j - 1, :, 2])

    laggard = np.minimum(St1 / NKY, St2 / SPX, St3 / HSI)

    # Calculate redemption
    redemption = np.zeros(M)

    # Compute knock-out date (if no knock-out event, knock-out date = N+1)
    KO_date = np.full(shape=M, fill_value=N + 1)
    for j in range(N, obs_days - 1, -obs_days):
        KO_date[laggard[j] >= KO] = j

    # Compute knock-out redemption
    KO_mask = (KO_date != (N + 1))
    redemption[KO_mask] += note_D * np.exp(-rf_rate('SPX', 504) * KO_date[KO_mask] / 252)

    # Compute knock-in redemption
    KI_mask = np.any(laggard <= KI, axis=0)
    redemption[~KO_mask] += note_D * np.minimum(1, laggard[-1][~KO_mask] * KI_mask[~KO_mask]) * np.exp(
        -rf_rate('SPX', 504) * T)  # knock-in event
    redemption[~KO_mask] += note_D * ~KI_mask[~KO_mask] * np.exp(-rf_rate('SPX', 504) * T)  # no knock-in event

    # Find upper bound for Bisection method
    CS = 0
    while True:
        coupon = np.zeros(M)
        for j in range(obs_days, N + 1, obs_days):
            coupon[KO_date >= j] += ((laggard[j] >= CS)[KO_date >= j] * note_D * max_CR * np.exp(-0.05 * j / 252))
            coupon[KO_date >= j] += ((laggard[j] < CS)[KO_date >= j] * note_D * min_CR * np.exp(-0.05 * j / 252))
        note0 = redemption + coupon
        E = np.sum(note0) / M
        if E / note_D < target:
            break
        elif CS > 100:
            print('No matter how high CS is, the note value will never be as low as 98% of the issue price.')
            quit()
        CS += 1

    # Bisection method
    a = 0
    b = CS
    while True:
        CS = (a + b) / 2
        coupon = np.zeros(M)
        for j in range(obs_days, N + 1, obs_days):
            coupon[KO_date >= j] += ((laggard[j] >= CS)[KO_date >= j] * note_D * max_CR
                                     * np.exp(-rf_rate('SPX', j) * j / 252))
            coupon[KO_date >= j] += ((laggard[j] < CS)[KO_date >= j] * note_D * min_CR
                                     * np.exp(-rf_rate('SPX', j) * j / 252))
        note0 = redemption + coupon
        E = np.sum(note0) / M
        if abs(E / note_D - target) < 0.0001:
            break
        if (E / note_D > target):
            a = CS
        else:
            b = CS

    print("Coupon strike is {0}%".format(np.round(CS, 4) * 100))

Volatility Surface

In [4]:
def BS(type, S, K, r, q, vol, T):
    d1 = (np.log(S/K) + (r-q+vol**2/2)*T)/(vol*np.sqrt(T))
    d2 = d1 - vol*np.sqrt(T)
    if type == 'C':
        return S*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        return K*np.exp(-r*T)*norm.cdf(-d2) - S*np.exp(-q*T)*norm.cdf(-d1)


# Bisection method for Black-Scholes formula
def Bisection_BS(V, type, S, K, r, q, T, err):
    low = 0
    up = 1
    while BS(type, S, K, r, q, up, T) < V:
        up += 100
    while True:
        mid = (low+up)/2
        if BS(type, S, K, r, q, mid, T) > V:
            up = mid
        else:
            low = mid
        if abs(BS(type, S, K, r, q, mid, T)-V) < err:
            return mid
        elif mid < 0.01:
            return 0


# Input parameters and it returns Black-Scholes implied vol
def BS_IMV(df, S0):
    BS_IMV = []
    for i in range(df.shape[0]):
        T = df.iloc[i, df.columns.get_loc('Days')]/252
        V = df.iloc[i, df.columns.get_loc('Price')]
        K = df.iloc[i, df.columns.get_loc('Strike')]
        r = df.iloc[i, df.columns.get_loc('Interest Rate')]/100
        q = 0.03
        err = 0.01
        BS_IMV.append(Bisection_BS(V, "C", S0, K, r, q, T, err))
    return BS_IMV


# Calibration formula for volatility curve fitting
def Calibration_Formula(x, delta, kappa, gamma):
    tanh = np.tanh(kappa * x)
    return delta*tanh/kappa + gamma/2*(tanh/kappa)**2


# Calibration process
def Calibration(df):
    calibration = []
    for date in df['Maturity Date'].unique():
        df_temp = df.loc[df['Maturity Date'] == date]
        K_atm = min(df_temp['Moneyness'], key=lambda x: abs(x))
        vol_atm = df_temp.loc[df_temp['Moneyness'] == K_atm, 'BS IMV'].values[0]

        x = df_temp['Moneyness']

        y = df_temp['BS IMV'] ** 2 - vol_atm ** 2

        popt, _ = curve_fit(Calibration_Formula, x, y)
        calibration.append((date, vol_atm, popt))
    return calibration


# Main function
def Volatility_Surface(index, moneyness, time_to_maturity, cs_rate, call_data):
    # As of 01-Sep-2023
    NKY = 32710.62
    SPX = 4515.77
    HSI = 18382.06
    df = call_data

    # Choice of index
    if index == 'NKY':
        S0 = NKY
        cs_rate = cs_JP
        if time_to_maturity > 189:  # Prevent extrapolation of NKY vol surface
            time_to_maturity = 189
    elif index == 'SPX':
        S0 = SPX
        cs_rate = cs_US
    elif index == 'HSI':
        S0 = HSI
        cs_rate = cs_HK
    else:
        print('No such index!')
        exit()

    df['Price'] = (df['Bid'] + df['Ask']) / 2  # Calculate the mid price from bid and ask
    df = df.loc[df['Price'] != 0]  # Drop rows which have Price = 0
    df['Interest Rate'] = cs_rate(df['Days'])
    df['Moneyness'] = np.log(df['Strike'] / S0)
    df['BS IMV'] = BS_IMV(df, S0)
    df = df.loc[df['BS IMV'] != 0]

    # Data cleaning
    if index == 'SPX':
        df.loc[df['Maturity Date'] == '5/9/2023', 'BS IMV'] = (df.loc[df['Maturity Date'] == '5/9/2023', 'IMV'] / 100)

    if index == 'HSI':
        df.loc[df['Maturity Date'] == '1/9/2023', 'BS IMV'] = (df.loc[df['Maturity Date'] == '1/9/2023', 'IMV'] / 100)
        df = df.loc[~(df['BS IMV'] >= 1)]
        df = df.loc[df['BS IMV'] != 0]

    calibration = Calibration(df)

    vols = []
    days = []
    for i, date in enumerate(df['Maturity Date'].unique()):
        vol2_atm = calibration[i][1] ** 2
        delta, kappa, gamma = calibration[i][2]

        vol = np.sqrt(Calibration_Formula(moneyness, delta, kappa, gamma) + vol2_atm)
        day = df.loc[df['Maturity Date'] == date, 'Days'].values[0]
        vols.append(vol)
        days.append(day)

    cs_vol = CubicSpline(days, vols)
    return cs_vol(time_to_maturity)

Risk Free Rate

In [7]:
    df_JP = pd.read_csv('Run_Data/Risk Free Rate/JP_RF Rate_01092023.csv')
    df_US = pd.read_csv('Run_Data/Risk Free Rate/US_RF Rate_01092023.csv')
    df_HK = pd.read_csv('Run_Data/Risk Free Rate/HK_RF Rate_01092023.csv')
    cs_JP = CubicSpline(df_JP['Days'], df_JP['Yield'])
    cs_US = CubicSpline(df_US['Days'], df_US['Yield'])
    cs_HK = CubicSpline(df_HK['Days'], df_HK['Yield'])
df = pd.read_csv('Run_Data/Options/' + 'HSI' + '_Calls_01092023.csv')
print(Volatility_Surface('HSI', 0.5, 200, cs_JP, df))

0.5285263879082741
